# DistilBERT fine-tuning으로 감정 분석 모델 학습하기

이번 실습에서는 pre-trained된 DistilBERT를 불러와 이전 주차 실습에서 사용하던 감정 분석 문제에 적용합니다. 먼저 필요한 library들을 불러옵니다.

In [ ]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets
# tqdm : 반복문의 진행 상황을 시각적으로 표시하는 progress bar를 제공
# boto3 : Amazon Web Services (AWS) 에 접근하기 위한 SDK
# requests : HTTP 요청을 보내고 응답을 받는 라이브러리
# regex : 정규 표현식을 사용하여 텍스트를 처리하는 라이브러리
# sentencepiece : Google에서 개발한 SentencePiece 모델을 사용하여 텍스트를 subword 단위로 분할하는 라이브러리
# sacremoses : 자연어 처리에서 흔히 사용되는 토큰화, stemming, lemmatization 등의 기능을 제공하는 라이브러리
# datasets : Hugging Face에서 제공하는 Datasets 라이브러리로, 다양한 자연어 처리 데이터셋을 쉽게 다운로드 가능

그 후, 우리가 사용하는 DistilBERT pre-training 때 사용한 tokenizer를 불러옵니다.

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

# DistilBERT tokenizer -> GPT tokenizer
# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'openai-gpt')
tokenizer.pad_token = tokenizer.unk_token # padding token 추가


"""
Hugging Face Transformers 라이브러리에서 DistilBERT 모델의 토크나이저를 로드하는 코드.
토크나이저는 텍스트를 모델이 이해할 수 있는 숫자 시퀀스로 변환하는 역할.

용도:
텍스트 데이터를 DistilBERT 모델에 입력하기 전에 전처리하는 데 사용.
텍스트를 토큰으로 분할하고 각 토큰에 해당하는 ID를 부여.

"""

DistilBERT의 tokenizer를 불러왔으면 이제 `collate_fn`과 data loader를 정의합니다. 이 과정은 이전 실습과 동일하게 다음과 같이 구현할 수 있습니다.

In [ ]:
"""
요약:

IMDB 데이터셋을 로드하고, 텍스트 데이터를 DistilBERT 토크나이저로 처리하여
모델 입력에 적합한 형태로 변환하는 collate_fn 함수를 정의하고, 데이터 로더를 생성하는 코드.

용도:

IMDB 데이터셋을 Hugging Face Datasets 라이브러리를 사용하여 로드함.
collate_fn 함수를 정의하여 텍스트 데이터를 일괄 처리하고, 토크나이징, 패딩, 잘라내기를 수행하여 모델 입력에 적합한 텐서 형태로 변환함.
DataLoader를 사용하여 학습 및 테스트 데이터 로더를 생성함.

"""

# ds = load_dataset("stanfordnlp/imdb")
ds = load_dataset("fancyzhx/ag_news")


def collate_fn(batch):
  # del: max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  #texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  texts = torch.LongTensor(tokenizer(texts, padding=True).input_ids)
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)



이제 pre-trained DistilBERT를 불러옵니다. 이번에는 PyTorch hub에서 제공하는 DistilBERT를 불러봅시다.

In [ ]:
"""
요약:
Hugging Face Transformers 라이브러리에서 DistilBERT 모델을 로드하는 코드
torch.hub.load() 함수를 사용해서 미리 학습된 DistilBERT 모델을 가져오고 출력.

용도:
텍스트 분류 등 다양한 자연어 처리 작업에 사용할 수 있는 미리 학습된 DistilBERT 모델을 로드.
모델 아키텍처를 확인하고, 추가적인 fine-tuning 또는 수정을 위한 기반으로 사용할 수 있음.
"""
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
model

출력 결과를 통해 우리는 DistilBERT의 architecture는 일반적인 Transformer와 동일한 것을 알 수 있습니다.
Embedding layer로 시작해서 여러 layer의 Attention, FFN를 거칩니다.

---

이제 DistilBERT를 거치고 난 `[CLS]` token의 representation을 가지고 text 분류를 하는 모델을 구현합시다.

In [ ]:
from torch import nn

class TextClassifier(nn.Module):
  """
  GPT 모델의 경우, 입력 텍스트의 마지막 토큰이 텍스트 전체의 의미를 가장 잘 담고 있으므로,
  마지막 토큰의 representation을 사용하여 분류하는 것이 더 적합.

  .'. forward 메서드를 수정하여
  마지막 토큰의 representation을 사용하도록 변경
  """
  def __init__(self):
    super().__init__()

    # self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'openai-gpt')

    self.classifier = nn.Linear(768, 4) # 출력 차원 4

  def forward(self, x):
    x = self.encoder(x)['last_hidden_state']
    # x = self.classifier(x[:, 0])
    x = self.classifier(x[:, -1]) # 마지막 토큰의 representation 사용


    return x


model = TextClassifier()



위와 같이 `TextClassifier`의 `encoder`를 불러온 DistilBERT, 그리고 `classifier`를 linear layer로 설정합니다.
그리고 `forward` 함수에서 순차적으로 사용하여 예측 결과를 반환합니다.

---

다음은 마지막 classifier layer를 제외한 나머지 부분을 freeze하는 코드를 구현합니다.

In [ ]:
"""
요약
모델의 인코더 부분 가중치를 고정하여 학습되지 않도록 설정.

용도
전이 학습에서 모델의 인코더 부분은 이미 잘 학습되어 있으므로,
새 데이터에 맞게 마지막 분류기 레이어만 학습하기 위해 사용.
"""
for param in model.encoder.parameters():
  param.requires_grad = False

위의 코드는 `encoder`에 해당하는 parameter들의 `requires_grad`를 `False`로 설정하는 모습입니다.
`requires_grad`를 `False`로 두는 경우, gradient 계산 및 업데이트가 이루어지지 않아 결과적으로 학습이 되지 않습니다.
즉, 마지막 `classifier`에 해당하는 linear layer만 학습이 이루어집니다.
이런 식으로 특정 부분들을 freeze하게 되면 효율적으로 학습을 할 수 있습니다.

마지막으로 이전과 같은 코드를 사용하여 학습 결과를 확인해봅시다.

In [8]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model = model.to('cuda')
# loss_fn = nn.BCEWithLogitsLoss() # 이진 분류를 위한 손실 함수입니다. (주석 처리됨)
loss_fn = nn.CrossEntropyLoss() # 다중 클래스 분류를 위한 손실 함수를 설정합니다.


optimizer = Adam(model.parameters(), lr=lr) # Adam 옵티마이저를 생성하고, 모델의 파라미터와 학습률을 전달합니다.
n_epochs = 10 # 학습할 에포크 횟수를 10으로 설정합니다.

for epoch in range(n_epochs):               # 설정된 에포크 횟수만큼 반복합니다.
  total_loss = 0.                           # 에포크마다 총 손실을 0으로 초기화합니다.
  model.train()                             # 모델을 학습 모드로 설정합니다.
  for data in train_loader:                 # train_loader에서 배치 단위로 데이터를 가져옵니다.
    model.zero_grad()                       # 이전 배치의 그래디언트를 초기화합니다.
    inputs, labels = data                   # 배치 데이터를 입력과 레이블로 분리합니다.
    # inputs, labels = inputs.to('cuda'), labels.to('cuda').float() # 입력과 레이블을 GPU로 이동시킵니다.
    inputs, labels = inputs.to('cuda'), labels.to('cuda') # .float() 제거


    # preds = model(inputs)[..., 0]           # 모델에 입력을 넣어 예측값을 계산합니다.
    preds = model(inputs)

    loss = loss_fn(preds, labels)           # 예측값과 레이블을 사용하여 손실을 계산합니다.
    loss.backward()                         # 손실을 기반으로 역전파를 수행하여 그래디언트를 계산합니다.
    optimizer.step()                        # 계산된 그래디언트를 사용하여 모델의 가중치를 업데이트합니다.

    total_loss += loss.item()               # 배치 손실을 총 손실에 누적합니다.

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}") # 에포크 번호와 총 손실을 출력합니다.

Epoch   0 | Train Loss: 723720.899017334
Epoch   1 | Train Loss: 722869.1695251465
Epoch   2 | Train Loss: 722794.111541748
Epoch   3 | Train Loss: 722767.2051086426
Epoch   4 | Train Loss: 722811.3719329834
Epoch   5 | Train Loss: 722888.4775085449
Epoch   6 | Train Loss: 722723.2508544922
Epoch   7 | Train Loss: 723008.5843200684
Epoch   8 | Train Loss: 722803.3864746094
Epoch   9 | Train Loss: 722864.1385803223


In [9]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  model.eval()
  train_acc = accuracy(model, train_loader)
  test_acc = accuracy(model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.250 | Test acc: 0.250


Loss가 잘 떨어지고, 이전에 우리가 구현한 Transformer보다 더 빨리 수렴하는 것을 알 수 있습니다.